In [1]:
################################################################################
# Download dataset and model from google drive to local storage
################################################################################

!ls
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
import os.path

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# # dataset_training = drive.CreateFile({'id': '1oqu3B_0gwRtVe___H7AH8wwxGzt25e3N'})
# # dataset_training.GetContentFile("coco_training_1.zip")

# # dataset_validation = drive.CreateFile({'id': '19Vsr2lU_fQMD3HSJLfpd3E5GBXWRO9Jm'})
# # dataset_validation.GetContentFile("coco_validation_1.zip")
  
# dataset = drive.CreateFile({'id': '1Yo4xiohHHoRquswIda4yZP6QxHK3aWo_'})
# dataset.GetContentFile("places2_training_2.zip") 
  
# # dataset = drive.CreateFile({'id': '17sUNxu0Eq7-sGih46QOVO1E62L6sw4Dp'})
# # dataset.GetContentFile("places2_training_1.zip")

# dataset = drive.CreateFile({'id': '1zJVZMpxyHw7pScvnG-pVr5PDD9XbQqRd'})
# dataset.GetContentFile("places2_validation_1.zip")

# # dataset = drive.CreateFile({'id': '1QO4NdHG99U6TVswmy4j3lz7ITxRp8D1B'})
# # dataset.GetContentFile("faces_in_wild.zip")

# # dataset = drive.CreateFile({'id': '10zuh0FbTmgMF6lgUf_EHnP9CAdvaofxI'})
# # dataset.GetContentFile("celeba_training_2.zip")

# # dataset = drive.CreateFile({'id': '1WQ6h5jQ9B4RCN7dDg2ySR5FRkK1EMfEl'})
# # dataset.GetContentFile("celeba_validation_1.zip")

# model = drive.CreateFile({'id': '12T24din3lXrseomcsXAetJdiIm_2MQxc'})
# model.GetContentFile("colorization_model.hdf5")

# # model = drive.CreateFile({'id': '1YpCm6bho9fNCVgY9Bh8WBaXX5OQlFqI1'})
# # model.GetContentFile("pspnet.h5")


12th_gen.csv   adc.json			places2_training_2.zip	  sample_data
12th_gen.hdf5  colorization_model.hdf5	places2_validation_1.zip


In [2]:
################################################################################
# Import necessary libraries
################################################################################

# !pip install q keras==2.0.0
# !pip install --upgrade Pillow
import zipfile
from PIL import Image
from skimage.color import rgb2lab
import numpy as np
from skimage.transform import resize
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model, load_model
from keras.layers import Conv2D, concatenate, Input, UpSampling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, Callback
from keras import layers
from keras.backend import tf as ktf
import random
import os
import time

# config = ktf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
# k.tensorflow_backend.set_session(ktf.Session(config=config))

Using TensorFlow backend.


In [0]:
################################################################################
# Generator and its functions
################################################################################

# Returns numpy arrays l, a and b (w x h x 1)
def lab_img(img):
    if img.format != ("JPEG" or "JPG"):
        img = img.convert("RGB")
    img = rgb2lab(img)

    l = img[:, :, 0]
    a = img[:, :, 1]
    b = img[:, :, 2]

    l = (np.array(l) / 100)
    a = ((np.array(a) + 127) / 255) * 2 - 1
    b = ((np.array(b) + 128) / 255) * 2 - 1

    l = np.expand_dims(l, axis=2)
    a = np.expand_dims(a, axis=2)
    b = np.expand_dims(b, axis=2)

    return l, a, b


# Returns batch of x and y values packed together
def batch_images(index, batch_size, images_size, image_paths, imgs, trained_model, flip):
    x_batch = np.zeros((batch_size, images_size[1], images_size[0], 1))
    s_batch = np.zeros((batch_size, int(images_size[1] / 8), int(images_size[0] / 8), 1280))
    y_batch = np.zeros((batch_size, images_size[1], images_size[0], 2))
    for i in range(index, index + batch_size):
        with imgs.open(image_paths[i]) as img:
            img = Image.open(img)

            if img.size != images_size:
                img = img.resize(images_size)

#             s = predict_segmentation(img.convert("L").convert("RGB"), trained_model) 
            s = predict_classification(img.convert("L").convert("RGB"), trained_model)
            l, a, b = lab_img(img)

            y = np.concatenate((a, b), axis=2)

            x_batch[i - index] = l
            s_batch[i - index] = s
            y_batch[i - index] = y
            
            if flip:
                x = np.fliplr(l)
                s = np.fliplr(s)
                y = np.fliplr(y)

                x_batch[i - index] = x
                s_batch[i - index] = s
                y_batch[i - index] = y

    return x_batch, s_batch, y_batch


# Returns one-hot encoded segmentation object (w x h x 150)
def predict_segmentation(img, trained_model):
    # TODO: do this in my model
    data_mean = np.array([[[123.68, 116.779, 103.939]]])
    input_size = (473, 473)
    output_size = (32, 32)

    if img.size != input_size:
        img = img.resize(input_size)

    pixel_img = np.array(img)
    pixel_img = pixel_img - data_mean
    bgr_img = pixel_img[:, :, ::-1]
    segmented_img = trained_model.predict(np.expand_dims(bgr_img, axis=0))[0]
#     segmented_img = segmented_img * 2 - 1  (in case of tanh)
    if output_size != input_size:
          segmented_img = resize(segmented_img,
                                 (output_size[1], output_size[0], 150),
                                  mode="constant",
                                  preserve_range=True)
    return segmented_img

  
def predict_classification(img, trained_model):
    output_size = (32, 32)
    classification_img = trained_model.predict(np.expand_dims(np.array(img), axis=0))[0]
    classification_img = resize(classification_img,
                               (output_size[1], output_size[0], 1280),
                               mode="constant",
                               preserve_range=True)
    return classification_img


# Yields batches of x and y values
def generator_fn(batch_size, images_path, images_size, trained_model, flip=False, validation=False):
    with zipfile.ZipFile(images_path) as imgs:
        image_paths = imgs.infolist()
        n_images = len(image_paths)
        if validation:
            n_images = batch_size
#         print(n_images)
        i = 0
        while True:
            if i + batch_size >= n_images:
                i = 0
#             print("batch start index: " + str(i) + "   " + images_path)
            x, s, y = batch_images(i, batch_size, images_size, image_paths, imgs, trained_model, flip)
            i += batch_size
            yield [x, s], y


In [0]:
################################################################################
# Loading, training and saving model
################################################################################

class Interp(layers.Layer):

    def __init__(self, new_size, **kwargs):
        self.new_size = new_size
        super(Interp, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Interp, self).build(input_shape)

    def call(self, inputs, **kwargs):
        new_height, new_width = self.new_size
        resized = ktf.image.resize_images(inputs, [new_height, new_width],
                                          align_corners=True)
        return resized

    def compute_output_shape(self, input_shape):
        return tuple([None, self.new_size[0], self.new_size[1], input_shape[3]])

    def get_config(self):
        config = super(Interp, self).get_config()
        config['new_size'] = self.new_size
        return config


# Returns trained model instance
def load_trained_model(path):
    trained_model = load_model(path, custom_objects={'Interp': Interp})
    trained_model._make_predict_function()
    return trained_model


# Returns main model
def model_definition():
    grayscale_input = Input(shape=(None, None, 1))
    grayscale = Conv2D(64, (3, 3), padding="same", activation="relu", use_bias=True, strides=2)(grayscale_input)
    grayscale = Conv2D(128, (3, 3), padding="same", activation="relu", use_bias=True)(grayscale)
    grayscale = Conv2D(128, (3, 3), padding="same", activation="relu", use_bias=True, strides=2)(grayscale)
    grayscale = Conv2D(256, (3, 3), padding="same", activation="relu", use_bias=True)(grayscale)
    grayscale = Conv2D(256, (3, 3), padding="same", activation="relu", use_bias=True, strides=2)(grayscale)
    grayscale = Conv2D(512, (3, 3), padding="same", activation="relu", use_bias=True)(grayscale)
    grayscale = Conv2D(512, (3, 3), padding="same", activation="relu", use_bias=True)(grayscale)
    grayscale = Conv2D(256, (3, 3), padding="same", activation="relu", use_bias=True)(grayscale)

    segmentation_input = Input(shape=(None, None, 1280))

    merged = concatenate([grayscale, segmentation_input], axis=3)
    colorized = Conv2D(128, (3, 3), padding="same", activation="relu", use_bias=True)(merged)
    colorized = UpSampling2D()(colorized)
    colorized = Conv2D(64, (3, 3), padding="same", activation="relu", use_bias=True)(colorized)
    colorized = UpSampling2D()(colorized)
    colorized = Conv2D(32, (3, 3), padding="same", activation="relu", use_bias=True)(colorized)
    colorized = Conv2D(16, (3, 3), padding="same", activation="relu", use_bias=True)(colorized)
    colorized = Conv2D(2, (3, 3), padding="same", activation="tanh", use_bias=True)(colorized)
    colorized = UpSampling2D()(colorized)

    model = Model(inputs=[grayscale_input, segmentation_input], outputs=colorized)
    model.compile(loss="mse", optimizer="adam")
    return model

  
class Upload2Drive(Callback):
  def __init__(self, generation, n_epochs):
      self.generation = generation
      self.n_epochs = n_epochs
      self.model_checkpoint = drive.CreateFile({"title": str(self.generation) + "th_gen.hdf5",
                                              "parents": [{"kind": "drive#childList",
                                                           "id": "116i9g5Ee6_TyDd0EVLsL7l_EE24LV4Zo"}]})
      self.model_log = drive.CreateFile({"title": str(self.generation) + "th_gen.csv",
                                        "parents": [{"kind": "drive#childList",
                                                     "id": "1Gb601hJZ4fC_tcJ_rWC4-HLLYk6A8owi"}]})
  
  def on_epoch_end(self, epoch, logs=None):
      if epoch % 2 == 0 or epoch + 1 == self.n_epochs:
          self.model.save(str(self.generation) + "th_gen.hdf5", overwrite=True)
          self.model_checkpoint.SetContentFile(str(self.generation) + "th_gen.hdf5")
          self.model_log.SetContentFile(str(self.generation) + "th_gen.csv")

          if gauth.access_token_expired:
              gauth.Refresh()
          try:
              self.model_checkpoint.Upload()
              self.model_log.Upload()
          except:
              print("upload checkpoint failed")

# Returns list of used keras callbacks
def callbacks(generation, n_epochs):
    cb = list()
    cb.append(CSVLogger(str(generation) + "th_gen.csv"))
    cb.append(Upload2Drive(generation, n_epochs))
    return cb


# Returns the model after training it
def train_model(model, training_data_fn, validation_data_fn, n_epochs, steps_per_epoch, validation_steps, generation):
    model.fit_generator(training_data_fn,
                        epochs=n_epochs,
                        steps_per_epoch=steps_per_epoch,
                        callbacks=callbacks(generation, n_epochs),
                        validation_data=validation_data_fn,
                        validation_steps=validation_steps,
                        verbose=2,
                        max_queue_size=1)
    return model


In [5]:
################################################################################
# Control panel
################################################################################

current_directory = ""
generation = 12 # don't forget about this

batch_size = 64 # twice as much gets trained if flip=True
batches_per_epoch = 40
n_epochs = 30
batches_per_validation = 1
images_size = (256, 256)

training_path = "places2_training_2.zip"
validation_path = "places2_validation_1.zip"
# trained_model = load_trained_model("pspnet.h5")
trained_model = MobileNetV2(include_top=False)
trained_model._make_predict_function()
# model = model_definition()
model = load_model("colorization_model.hdf5")

training_data_fn = generator_fn(batch_size, training_path, images_size, trained_model)
validation_data_fn = generator_fn(batch_size, validation_path, images_size, trained_model, validation=True)

start_time = time.time()
train_model(model, 
            training_data_fn, 
            validation_data_fn, 
            n_epochs, 
            batches_per_epoch, 
            batches_per_validation, 
            generation)
print("Training took: " + str(time.time() - start_time))

!ls


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


Epoch 1/30
 - 482s - loss: 0.0116 - val_loss: 0.0114
Epoch 2/30
 - 452s - loss: 0.0117 - val_loss: 0.0117
Epoch 3/30
 - 460s - loss: 0.0115 - val_loss: 0.0114
Epoch 4/30
 - 444s - loss: 0.0118 - val_loss: 0.0114
Epoch 5/30
 - 460s - loss: 0.0113 - val_loss: 0.0114
Epoch 6/30
 - 454s - loss: 0.0116 - val_loss: 0.0113
Epoch 7/30
 - 461s - loss: 0.0116 - val_loss: 0.0113
Epoch 8/30
 - 455s - loss: 0.0115 - val_loss: 0.0114
Epoch 9/30
 - 460s - loss: 0.0114 - val_loss: 0.0112
Epoch 10/30
 - 456s - loss: 0.0114 - val_loss: 0.0113
Epoch 11/30
 - 459s - loss: 0.0116 - val_loss: 0.0112
Epoch 12/30
 - 455s - loss: 0.0117 - val_loss: 0.0113
Epoch 13/30
 - 461s - loss: 0.0113 - val_loss: 0.0113
Epoch 14/30
 - 453s - loss: 0.0113 - val_loss: 0.0113
Epoch 15/30
 - 461s - loss: 0.0109 - val_loss: 0.0112
Epoch 16/30
 - 456s - loss: 0.0115 - val_loss: 0.0113
Epoch 17/30
 - 459s - loss: 0.0106 - val_loss: 0.0114
Epoch 18/30
 - 455s - loss: 0.0114 - val_loss: 0.0113
Epoch 19/30
 - 460s - loss: 0.0110 - 